In [55]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from urllib.request import urlopen
import json


sns.set()
%matplotlib inline


In [56]:
df = pd.read_csv('FAO.csv', encoding="ISO-8859-1")

In [57]:
df.head()

,Area Abbreviation,Area Code,Area,Item Code,Item,Element Code,Element,Unit,latitude,longitude,...,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
0,AFG,2,Afghanistan,2511,Wheat and products,5142,Food,1000 tonnes,33.94,67.71,...,3249.0,3486.0,3704.0,4164.0,4252.0,4538.0,4605.0,4711.0,4810,4895
1,AFG,2,Afghanistan,2805,Rice (Milled Equivalent),5142,Food,1000 tonnes,33.94,67.71,...,419.0,445.0,546.0,455.0,490.0,415.0,442.0,476.0,425,422
2,AFG,2,Afghanistan,2513,Barley and products,5521,Feed,1000 tonnes,33.94,67.71,...,58.0,236.0,262.0,263.0,230.0,379.0,315.0,203.0,367,360
3,AFG,2,Afghanistan,2513,Barley and products,5142,Food,1000 tonnes,33.94,67.71,...,185.0,43.0,44.0,48.0,62.0,55.0,60.0,72.0,78,89
4,AFG,2,Afghanistan,2514,Maize and products,5521,Feed,1000 tonnes,33.94,67.71,...,120.0,208.0,233.0,249.0,247.0,195.0,178.0,191.0,200,200


In [58]:
df.isnull().sum()

Area Abbreviation      0
Area Code              0
Area                   0
Item Code              0
Item                   0
                    ... 
Y2009                104
Y2010                104
Y2011                104
Y2012                  0
Y2013                  0
Length: 63, dtype: int64

In [59]:
df.dropna(inplace=True)

In [60]:
df.isnull().sum()

Area Abbreviation    0
Area Code            0
Area                 0
Item Code            0
Item                 0
                    ..
Y2009                0
Y2010                0
Y2011                0
Y2012                0
Y2013                0
Length: 63, dtype: int64

In [61]:
df.duplicated().sum()

0

In [62]:
df.shape

(17938, 63)

In [63]:
save_as = 'cleaned-data.csv'
df.to_csv(save_as, index=False)

In [64]:
gb = df.groupby(['Area Abbreviation'])['Y2013'].sum()
new_df = pd.DataFrame(gb)
new_df = new_df.reset_index()
new_df

,Area Abbreviation,Y2013
0,AFG,23007
1,AGO,48639
2,ALB,8271
3,ARE,15444
4,ARG,80843
...,...,...
138,WSM,337
139,YEM,18745
140,ZAF,70240
141,ZMB,10996


In [78]:
# fig = px.choropleth(new_df, locations="Area Abbreviation",
#                     color="Y2013",
#                     hover_name="Area Abbreviation",
#                     color_continuous_scale=px.colors.sequential.Plasma)
# fig.show()

In [66]:
total_food = new_df['Y2013'].sum()

In [67]:
# new_df['%'] = new_df['Y2013'] / total_food
# new_df.head()

# wanted_mean = total_food/new_df['Y2013'].count()
# wanted_mean

In [68]:
# fig = px.choropleth(new_df, locations="Area Abbreviation",
#                     color="%",
#                     hover_name="Area Abbreviation",
#                     color_continuous_scale=px.colors.sequential.Plasma)
# fig.show()

In [69]:
df2 = pd.read_csv('data2.csv')
df2.head()

,Country,2013,Area Abbreviation
0,Albania,5.2,ALB
1,Algeria,<5,DZA
2,Angola,19.1,AGO
3,Argentina,<5,ARG
4,Bangladesh,19.4,BGD


In [70]:
df2.isna().sum()

Country              0
2013                 0
Area Abbreviation    0
dtype: int64

In [72]:
cond = df2['2013'] == '<5'
df2['2013'] = np.where(cond, 5, df2['2013'])
df2['2013'] = np.where(cond, 5, df2['2013'].astype(float))

In [82]:
df_combined = pd.merge(new_df, df2, on='Area Abbreviation', how='outer')
df_combined
df_combined.dropna(inplace=True)
df_combined.head()


,Area Abbreviation,Y2013,Country,2013
1,AGO,48639.0,Angola,19.1
2,ALB,8271.0,Albania,5.2
4,ARG,80843.0,Argentina,5.0
9,BEN,16481.0,Benin,13.3
10,BFA,18980.0,Burkina Faso,22.2


In [108]:
# Shows the amount of food produced by each country
fig = px.choropleth(df_combined, locations="Area Abbreviation",
                    color="Y2013",
                    hover_name="Country",
                    color_continuous_scale=px.colors.sequential.Blugrn)
fig.show()

In [109]:
# Shows the amount of GHI by each country
fig = px.choropleth(df_combined, locations="Area Abbreviation",
                    color="2013",
                    hover_name="Country",
                    color_continuous_scale=px.colors.sequential.Blugrn)
fig.show()

In [85]:
df_combined.head()

,Area Abbreviation,Y2013,Country,2013
1,AGO,48639.0,Angola,19.1
2,ALB,8271.0,Albania,5.2
4,ARG,80843.0,Argentina,5.0
9,BEN,16481.0,Benin,13.3
10,BFA,18980.0,Burkina Faso,22.2


In [117]:
total = df_combined['Y2013'].sum() / len(df_combined)
total

96160.92045454546

In [118]:
cond = df_combined['2013'] <= 10

df_combined['food secure'] = np.where(cond, True, False)
df_combined.head()

,Area Abbreviation,Y2013,Country,2013,food secure,results
1,AGO,48639.0,Angola,19.1,False,2
2,ALB,8271.0,Albania,5.2,True,4
4,ARG,80843.0,Argentina,5.0,True,1
9,BEN,16481.0,Benin,13.3,False,2
10,BFA,18980.0,Burkina Faso,22.2,False,2


In [119]:
cond1 = df_combined['food secure'] == True
cond2 = df_combined['Y2013'] >=96160
df_combined['results'] = np.where(cond1 & cond2, 1, 0)
df_combined.head()

,Area Abbreviation,Y2013,Country,2013,food secure,results
1,AGO,48639.0,Angola,19.1,False,0
2,ALB,8271.0,Albania,5.2,True,0
4,ARG,80843.0,Argentina,5.0,True,0
9,BEN,16481.0,Benin,13.3,False,0
10,BFA,18980.0,Burkina Faso,22.2,False,0


In [120]:
cond1 = df_combined['food secure'] == False
cond2 = df_combined['Y2013'] < 96160
df_combined['results'] = np.where(cond1 & cond2, 2, df_combined['results'])
df_combined.head()

,Area Abbreviation,Y2013,Country,2013,food secure,results
1,AGO,48639.0,Angola,19.1,False,2
2,ALB,8271.0,Albania,5.2,True,0
4,ARG,80843.0,Argentina,5.0,True,0
9,BEN,16481.0,Benin,13.3,False,2
10,BFA,18980.0,Burkina Faso,22.2,False,2


In [121]:
cond1 = df_combined['food secure'] == True
cond2 = df_combined['Y2013'] < 96160
df_combined['results'] = np.where(cond1 & cond2, 4, df_combined['results'])
df_combined.head()

,Area Abbreviation,Y2013,Country,2013,food secure,results
1,AGO,48639.0,Angola,19.1,False,2
2,ALB,8271.0,Albania,5.2,True,4
4,ARG,80843.0,Argentina,5.0,True,4
9,BEN,16481.0,Benin,13.3,False,2
10,BFA,18980.0,Burkina Faso,22.2,False,2


In [122]:
cond1 = df_combined['food secure'] == False
cond2 = df_combined['Y2013'] >= 96160
df_combined['results'] = np.where(cond1 & cond2, 3, df_combined['results'])
df_combined.head()

,Area Abbreviation,Y2013,Country,2013,food secure,results
1,AGO,48639.0,Angola,19.1,False,2
2,ALB,8271.0,Albania,5.2,True,4
4,ARG,80843.0,Argentina,5.0,True,4
9,BEN,16481.0,Benin,13.3,False,2
10,BFA,18980.0,Burkina Faso,22.2,False,2


In [123]:
# Shows which ones fit our assum vs which ones dont 
fig = px.choropleth(df_combined, locations="Area Abbreviation",
                    color="results",
                    hover_name="Country",
                    color_continuous_scale=px.colors.sequential.Plasma)
fig.show()

In [124]:
cond1 = df_combined['results'] == 3
cond2 = df_combined['results'] == 4
new_df = df_combined[cond1 | cond2]
new_df.head()

,Area Abbreviation,Y2013,Country,2013,food secure,results
2,ALB,8271.0,Albania,5.2,True,4
4,ARG,80843.0,Argentina,5.0,True,4
11,BGD,120854.0,Bangladesh,19.4,False,3
12,BGR,12335.0,Bulgaria,5.0,True,4
23,CHL,32900.0,Chile,5.0,True,4


In [125]:
#Surprise map - Shows which dont fit our assum and maps by amount of food
fig = px.choropleth(new_df, locations="Area Abbreviation",
                    color="Y2013",
                    hover_name="Country",
                    color_continuous_scale=px.colors.sequential.Blugrn)
fig.show()